# Pràctica 1 - Xarxes Neuronals i Deep Learning

#### Document realitzat pels alumnes Pau Prat i Violeta Bonet, del grau en Inteligència Artificial (UPC).

En primer lloc, cal destacar que aquesta primera cel·la conté tots els mòduls i llibreries necessàries per a l'execució del codi. 

Per tant, en cas de no tenir-les, caldrà executar aquesta cel·la prèviament.

In [ ]:
'''
%pip install --upgrade pip  --quiet
%pip install pandas  --upgrade --quiet
%pip install numpy  --upgrade --quiet
%pip install scipy  --upgrade --quiet
%pip install statsmodels  --upgrade --quiet
%pip install seaborn  --upgrade --quiet
%pip install scikit-learn==1.3.0
%pip install tqdm ipykernel matplotlib ipywidgets --upgrade --quiet   
%pip install plotly numpy==1.25 nbformat umap-learn
%pip install ucimlrepo
%pip install mlxtend
%pip install pydotplus
%pip install imbalanced-learn
%pip install yellowbrick
%pip install missingno
%pip install tensorflow
%load_ext autoreload
'''

Imports bàsics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Definim la mida predeterminada dels gràfics

In [ ]:
plt.rcParams['figure.figsize'] = [9, 6]  

plt.rcParams['font.size'] = 14

sns.set(font_scale=1)

# 1. Anàlisi Exploratòria de Dades (EDA)

#### <span style="color:lightgreen"> Carreguem la base de dades</span>

In [ ]:
df = pd.read_csv("smartphone_data.csv")

df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
def classify_features(df, target): 
    initial_numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    initial_categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_features = []
    categorical_features = initial_categorical_features  

    for column in initial_numerical_features:
        if column != target:
            if df[column].nunique() < 10:
                categorical_features.append(column) 
            else:
                numerical_features.append(column)  

    def feature_type(column):
        if column in numerical_features:
            return 'Numerical'
        elif column in categorical_features:
            return 'Categorical'
        else:
            return 'Boolean'

    features = pd.DataFrame({
        'Feature': [column for column in df.columns if column != 'price'],
        'Type': [df[column].dtype for column in df.columns if column != 'price'],
        'Unique values': [df[column].nunique() for column in df.columns if column != 'price'],
        'Category': [feature_type(column) for column in df.columns if column != 'price']
    })

    features.sort_values(by='Unique values', ascending=True, inplace=True)
    return numerical_features, categorical_features, features

numerical_features, categorical_features, features = classify_features(df, 'price')

features

In [ ]:
for variable in ['has_5g', 'has_nfc', 'has_ir_blaster']:
    df[variable] = df[variable].map({True: 1, False: 0})

In [ ]:
numerical_features, categorical_features, features = classify_features(df, 'price')

features

In [ ]:
# Eliminar 'model' de la llista de variables categòriques i de 'df'
categorical_features.remove('model')
df.drop('model', axis=1, inplace=True)

#### <span style="color:lightgreen"> Visualitzem la distribució de cada variable numèrica</span>

In [ ]:
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize = (12, 6))
axes = axes.flatten()

for i, ax in enumerate(axes[:len(numerical_features)]):
    sns.histplot(df[numerical_features[i]], bins = 30, color = 'blue', edgecolor = 'white', kde = True, ax = ax)

# Eliminar els subplots sobrants
for ax in axes[len(numerical_features):]:
    fig.delaxes(ax)

plt.tight_layout()
#plt.savefig('./plots/num_dist.png')

En cas que es vulgui obtenir el gràfic de la distribució de cada variable numèrica per separat, simplement cal descomentar i executar la següent cel·la.

In [ ]:
'''for feature in numerical_features:
    mean = df[feature].mean()
    fig, ax = plt.subplots()
    sns.histplot(df[feature], kde=False, ax=ax, edgecolor="black")
    ax.plot([mean], [-0.6], marker='^', markersize=9, color="red")
    ax.set_title(f'Distribució de {feature}')
    ax.set_xlabel(feature, size=10)
    ax.set_ylabel("Freqüència", size=10)
    plt.tight_layout()
    #plt.savefig(f'./plots/{feature}_distribution.png')'''

#### <span style="color:lightgreen"> Histograma de la freqüència per classe de cada variable categòrica </span>

In [ ]:
fig, axes = plt.subplots(4, 4, figsize = (16, 16))
axes = axes.flatten()

for i, ax in enumerate(axes):
    sns.countplot(data = df, x = categorical_features[i], color = 'green', ax = ax)
    if categorical_features[i] in ['resolution', 'processor_brand', 'brand_name']:
        ax.set_xticklabels([])  

plt.tight_layout()
#plt.savefig('./plots/hist_freq.png')

En cas que es vulgui obtenir el gràfic de la freqüència per classe de cada variable categòrica per separat, simplement cal descomentar i executar la següent cel·la.

In [ ]:
'''for feature in categorical_features:
    plt.figure()
    df[feature].value_counts().plot(kind='bar', color='green')
    plt.title(f'Freqüència per classe - {feature}')
    if feature == 'resolution':
        plt.xticks(rotation=90, fontsize=8)  
    else:
        plt.xticks(rotation=70, fontsize=10)
    plt.tight_layout()
    #plt.savefig(f'./plots/{feature}_frequency.png')'''

#### <span style="color:lightgreen"> Correlacions entre variables numèriques </span>

In [ ]:
corr_matrix = df[numerical_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.tight_layout()  
#plt.savefig('./plots/correlations_heatmap.png')
plt.show()

#### <span style="color:lightgreen"> Correlació entre variables categòriques i variable objectiu </span>

Convertim les unitats de 'price' de rúpies a euros

In [ ]:
# Multiplicar els valors de price per 0.011
df['price'] = df['price'] * 0.011

In [ ]:
# Visualitzar la distribució de 'price'
plt.figure()
sns.histplot(df['price'], bins = 30, color = 'blue', edgecolor = 'white', kde = True)
plt.title('Distribució de price')
plt.xlabel('Price')
plt.ylabel('Freqüència')
plt.tight_layout()
#plt.savefig('./plots/price_distribution.png')

In [ ]:
for i, feature in enumerate(categorical_features):
    plt.figure(figsize=(10, 5))
    sns.boxplot(x=feature, y='price', data=df)
    plt.title(f'Distribució de price segons {feature}')
    plt.xticks(rotation=90)
    plt.tight_layout()  
    #plt.savefig(f'./plots/boxplots/{feature}.png')  


#### <span style="color:lightgreen"> Correlació entre variables numèriques i variable objectiu </span>

In [ ]:
sns.set(style="whitegrid")

num_rows = 2
num_cols =  4

fig, axes = plt.subplots(num_rows, num_cols, figsize = (14, 7))
axes = axes.flatten()

for i, feature in enumerate(numerical_features):
    sns.regplot(x=feature, y='price', data=df, scatter_kws={'alpha':0.5}, line_kws={"color": "red"}, ax=axes[i])  # alpha para transparencia de puntos
    axes[i].set_title(f'Relació entre Price i {feature}')  
    axes[i].set_xlabel(feature) 
    axes[i].set_ylabel('Price')  

# Eliminar els subplots sobrants
for ax in axes[len(numerical_features):]:
    fig.delaxes(ax)

plt.tight_layout()
#plt.savefig('./plots/numerical_correlations.png')
plt.show()

# 2. Preprocessament

#### <span style="color:lightgreen"> Missings </span>

In [ ]:
import missingno as msno

plt.figure()
msno.matrix(df)
plt.tight_layout()  
#plt.savefig('./plots/missingno_matrix.png')  

In [ ]:
def missing_data(data):
    total_missing = data.isna().sum().sort_values(ascending=False)
    percent_missing = round(100 * (data.isnull().sum() / len(data)), 2).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Total Missing': total_missing, 'Percent Missing (%)': percent_missing})
    return missing_data
missing_data(df)

Eliminem 'extended_upto' ja que té gairebé un 50% de missings

In [ ]:
categorical_features.remove('extended_upto')
df = df.drop('extended_upto', axis=1)

Els altres missings els imputarem un cop particionem el dataset en train i test

#### <span style="color:lightgreen"> Outliers </span>

In [ ]:
for feature in numerical_features:
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot boxplot
    sns.boxplot(x=df[feature], ax=axes[0])
    axes[0].set_title(f'{feature} with outliers')

    # Plot distribution
    sns.histplot(data=df, x=feature, kde=True, ax=axes[1])
    axes[1].set_title(f'{feature} Distribution')

    plt.tight_layout()
    #fig.savefig(f'./plots/dist_with_outliers/{feature}_with_outliers.png')

Eliminarem els outliers seguint el criteri del Rang Interquartil, excepte la variable objectiu 'price'

Per tant, considerarem outliers:
* Els valors més grans que Q1 - 1.5*IQR
* Els valors més petits que Q3 + 1.5*IQR


In [ ]:
for feature in numerical_features:
    # Calcular IQR
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Definir el límit inferior i superior
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with NaN
    df.loc[(df[feature] < lower_bound) | (df[feature] > upper_bound), feature] = np.nan

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot boxplot
    sns.boxplot(x=df[feature], ax=axes[0])
    axes[0].set_title(f'{feature} Boxplot')

    # Plot distribution
    sns.histplot(data=df, x=feature, kde=True, ax=axes[1])
    axes[1].set_title(f'{feature} Distribution')

    plt.tight_layout()
    #fig.savefig(f'./plots/dist_without_outliers/{feature}_without_outliers.png')

In [ ]:
# Eliminar files tal que 'price' es un outlier
Q1 = df['price'].quantile(0.10)
Q3 = df['price'].quantile(0.90)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

df.shape

Ara, tornem a observar la distribució de 'price' i podem comprovar que els valors ja són més típics

In [ ]:
# Visualitzar la distribució de 'price'
plt.figure()
sns.histplot(df['price'], bins = 30, color = 'blue', edgecolor = 'white', kde = True)
plt.title('Distribució de price')
plt.xlabel('Price')
plt.ylabel('Freqüència')
plt.tight_layout()
#plt.savefig('./plots/price_distribution.png')

# 3. Remostreig

#### <span style="color:lightgreen"> Partició del dataset en Train i Test </span>

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1) 
y = df['price']

# Dividir el dataset en train y test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [ ]:
# Mostrar el número d'observacions y features de cada set
sets_df = pd.DataFrame(columns=['Set', 'Number of Observations', 'Number of Features'])
sets_df.loc[len(sets_df)] = ['Train', X_train.shape[0], X_train.shape[1]]
sets_df.loc[len(sets_df)] = ['Test', X_test.shape[0], X_test.shape[1]]

sets_df

#### <span style="color:lightgreen"> Imputació de Missings</span>

##### Variables numèriques

In [ ]:
missing_before_num = X_train[numerical_features].isnull().sum()
mean_before = X_train[numerical_features].mean()
stderr_before = X_train[numerical_features].sem()
median_before = X_train[numerical_features].median()

In [ ]:
from sklearn.impute import KNNImputer

# Crear l'imputador KNN
imputer = KNNImputer(n_neighbors=5)

# Ajustar l'imputador a les característiques numèriques de les dades d'entrenament i transformar-les
X_train[numerical_features] = imputer.fit_transform(X_train[numerical_features])

# Transformar les característiques numèriques de les dades de prova utilitzant l'imputador ajustat
X_test[numerical_features] = imputer.transform(X_test[numerical_features])

In [ ]:
missing_after_num = X_train[numerical_features].isnull().sum()
mean_after = X_train[numerical_features].mean()
stderr_after = X_train[numerical_features].sem()
median_after = X_train[numerical_features].median()

In [ ]:
stats_comparison_num = pd.DataFrame({
    'Feature': missing_before_num.index, 
    'Mean (old)': mean_before.values,
    'Mean': mean_after.values,
    'Std_Error (old)': stderr_before.values,
    'Std_Error': stderr_after.values,
    'Median (old)': median_before.values,
    'Median': median_after.values
})
stats_comparison_num

##### Variables categòriques

In [ ]:
# Guardar les dades originals
original_data = X_train.copy()

In [ ]:
# Llista amb els noms de les variables categòriques que tenen almenys un missing
cat_features_missings = [feature for feature in categorical_features if X_train[feature].isnull().any()]
print(cat_features_missings)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

X_train[categorical_features] = imputer.fit_transform(X_train[categorical_features])
X_test[categorical_features] = imputer.transform(X_test[categorical_features])

In [ ]:
for feature in cat_features_missings:
    plt.figure(figsize=(10, 4))

    # Abans de la imputació (excloïm missings)
    plt.subplot(1, 2, 1)
    filtered_data = original_data[original_data[feature].notna()]
    sns.countplot(data=filtered_data, x=feature, color='green')
    plt.title(f'{feature} - Original')
    plt.xticks(rotation=60)  

    # Després de la imputació
    plt.subplot(1, 2, 2)
    sns.countplot(data=X_train, x=feature, color='green')
    plt.title(f'{feature} - Després de la Imputació')
    plt.xticks(rotation=60)  
    plt.tight_layout()  
    #plt.savefig(f'./plots/dist_moda/{feature}_moda.png')

#### <span style="color:lightgreen"> Recodificació de variables categòriques </span>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Crear una còpia dels conjunts de dades per no modificar els originals
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# Crear el codificador
le = LabelEncoder()

# Obtindre totes les categories úniques per a 'brand_name' i 'processor_brand'
all_brands = pd.concat([X_train['brand_name'], X_test['brand_name']]).dropna().unique()
all_processors = pd.concat([X_train['processor_brand'], X_test['processor_brand']]).dropna().unique()

# One-Hot Encoding per 'brand_name' i 'processor_brand'
for col, all_categories in zip(['brand_name', 'processor_brand'], [all_brands, all_processors]):
    dummies = pd.get_dummies(pd.concat([X_train[col], X_test[col]], axis=0), prefix=col, drop_first=True)
    X_train_encoded = pd.concat([X_train_encoded, dummies.loc[X_train.index]], axis=1)
    X_test_encoded = pd.concat([X_test_encoded, dummies.loc[X_test.index]], axis=1)
    X_train_encoded.drop([col], axis=1, inplace=True)
    X_test_encoded.drop([col], axis=1, inplace=True)

# Codificar les altres columnes categòriques de tipus object
for col in categorical_features:
    if col not in ['brand_name', 'processor_brand'] and X_train_encoded[col].dtype == 'object':
        le.fit(pd.concat([X_train_encoded[col], X_test_encoded[col]])) 
        X_train_encoded[col] = le.transform(X_train_encoded[col])
        X_test_encoded[col] = le.transform(X_test_encoded[col])

In [ ]:
X_train_encoded.shape

#### <span style="color:lightgreen"> Normalització de variables numèriques </span>

In [ ]:
from sklearn.preprocessing import StandardScaler

# Crear una còpia dels conjunts de dades per no modificar els originals
X_train_normalized = X_train_encoded.copy()
X_test_normalized = X_test_encoded.copy()

# Crear el normalitzador
scaler = StandardScaler()

# Normalitzar les columnes numèriques
X_train_normalized[numerical_features] = scaler.fit_transform(X_train_normalized[numerical_features])
X_test_normalized[numerical_features] = scaler.transform(X_test_normalized[numerical_features])

In [ ]:
for feature in numerical_features:
    plt.figure(figsize=(10, 4))
    X_train_normalized[feature].plot.hist()
    plt.title(f'Distribució Normalitzada - {feature}')
    #plt.savefig(f'./plots/normalized/{feature}_normalized.png')

#### <span style="color:lightgreen"> CV per avaluar el model </span>

# 4.1 Model Lineal Base

#### <span style="color:lightgreen"> Entrenament i avaluació d'un model de regressió lineal </span>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Crea el model de regressió lineal
model = LinearRegression()

# Entrena el model
model.fit(X_train_normalized, y_train)

# Fes prediccions amb les dades de prova
y_pred = model.predict(X_test_normalized)

# Avaluar el model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

#### <span style="color:lightgreen"> Interpretació dels resultats obtinguts (mètriques de classificació, coeficients, etc.) </span>

In [ ]:
def linearity(model, X, y):
    """
    Funció per visualitzar la linealitat entre els valors predits i els reals.
    """
    # Generar prediccions
    y_pred = model.predict(X)

    # Crear un gràfic de dispersió dels valors reals vs. predits
    plt.figure(figsize=(8, 8))
    plt.scatter(y, y_pred, alpha=0.5)
    plt.title('Comparació entre els Valors Reals i els Predits')
    plt.xlabel('Valors Reals')
    plt.ylabel('Valors Predits')

    # Dibuixar la línia diagonal que representa la perfecta predicció
    max_val = max(np.max(y), np.max(y_pred))
    min_val = min(np.min(y), np.min(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--')

    plt.grid(True)
    plt.show()
    
linearity(model, X_test_normalized, y_test)

# 4.2 EXTRA

### Trobar el millor model per aquest problema de regressió mitjançant validació creuada

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Crear un diccionari de models a avaluar
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(max_iter=10000),
    'Lasso Regression': Lasso(max_iter=10000),
    'Random Forest': RandomForestRegressor(),
    'K-Nearest Neighbors': make_pipeline(StandardScaler(), KNeighborsRegressor()) 
}

# Configuració de la validació creuada
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Avaluar cada model fent servir el R2 Score
results = {}
for name, model in models.items():
    cv_results = cross_val_score(model, X_train_normalized, y_train, cv=kf, scoring='r2')
    results[name] = cv_results.mean()  

# Trobar el model que dona un millor resultat de R2
best_model = max(results, key=results.get)
print("Millor model:", best_model)
print("R2-score de cada model:", results)


In [ ]:
'''from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Definir el model
model = RandomForestRegressor(random_state=42)

# Definir els hiperparàmetres a provar
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Crear el Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, scoring='r2', verbose=2, n_jobs=-1)

# Ajustar el Grid Search als dades
grid_search.fit(X_train_normalized, y_train)

# Imprimir els millors hiperparàmetres trobats
print(f'Millors hiperparàmetres: {grid_search.best_params_}')
print(f'Millor score: {grid_search.best_score_}')'''

El codi anterior té un cost computacional molt elevat, per tant, com que els resultats són reproductibles (random_state), els hiperparàmetres òptims obtinguts són els següents:
-  {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}

En cas que es vulgui executar la cel·la per tal de comprovar-ho, simplement cal descomentar-la.

# 5. Perceptró Multicapa

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, fashion_mnist

from keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_covtype, fetch_california_housing
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score

In [ ]:
# Funció per construir el model
def build_model(n_layers, n_units, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(n_units, activation='relu', input_shape=(X_train_normalized.shape[1],)))
    for _ in range(n_layers - 1):
        model.add(Dense(n_units, activation='relu'))
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Funció per a visualitzar les corbes d'aprenentatge
def plot_curves(history):
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Curves')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mean_squared_error'], label='Train MSE')
    plt.plot(history.history['val_mean_squared_error'], label='Validation MSE')
    plt.title('MSE Curves')
    plt.legend()
    plt.show()

# Procés iteratiu
n_iterations = 4
n_units = 64
batch_sizes = [64, 32, 16, 8]

for i in range(n_iterations):
    print(f"Iteració {i+1}")
    model = build_model(i + 1, n_units, learning_rate=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    history = model.fit(X_train_normalized, y_train, epochs=1000, batch_size=batch_sizes[i], validation_split=0.1, verbose=2, callbacks=[early_stopping])
    
    # Evalua el model
    loss, mse = model.evaluate(X_test_normalized, y_test, verbose=1)
    y_pred = model.predict(X_test_normalized)
    r2 = r2_score(y_test, y_pred.flatten())
    print(f"Test Loss: {loss}, Test MSE: {mse}, R2 Score: {r2}")
    
    # Visualitza les corbes d'aprenentatge
    plot_curves(history)


In [ ]:
'''# Defineix l'arquitectura del model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_normalized.shape[1],)), 
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Capa de sortida per a regressió
])

# Defineix l'optimitzador i la taxa d'aprenentatge
learning_rate = 0.001  # Taxa més petita per a regressió, per promoure una convergència més suau
optimizer = Adam(learning_rate=learning_rate)

# Compila el model
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrena el model
num_epochs = 1000
batch_size = 32  # Batch size més petit per millorar la generalització
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  # Early stopping per evitar overfitting
history = model.fit(X_train_normalized, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.1, verbose=2, callbacks=[early_stopping])

# Avalua el model
loss, mse = model.evaluate(X_test_normalized, y_test)
print("Test Loss:", loss)
print("Test MSE:", mse)

# Fer prediccions amb les dades de prova
y_pred = model.predict(X_test_normalized)

# Calcular i imprimir el R2 score
r2 = r2_score(y_test, y_pred.flatten())  # Assegurar-se que les dimensions coincideixen
print(f"R2 Score: {r2}")

# Funció per a visualitzar les corbes d'aprenentatge
def plot_curves(history):
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Curves')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mean_squared_error'], label='Train MSE')
    plt.plot(history.history['val_mean_squared_error'], label='Validation MSE')
    plt.title('MSE Curves')
    plt.legend()
    plt.show()

# Visualitza les corbes d'aprenentatge
plot_curves(history)'''